In [1]:
from dataloader import *
from model import *
from helper import generate_report

In [2]:
data = DataLoader(config_mixed_flickr30k_coco17)

Photos: train=29000
Loaded all descriptions from train dataset: 31014 
Descriptions: train=29000
Found 400000 word vectors.
Photos: test=5000
preprocessed words 18015 -> 5261


In [3]:
model=ModelImpl(data)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 80)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 80, 199)      1047138     input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_1[0][0]                    
______________________________________________________________________________________________

In [4]:
model.train()

Epoch 1/2



KeyboardInterrupt



In [ ]:
model.evaluate()

In [3]:
generate_report(data.config["results_directory"])


 Final results saved to final_results.csv
